In [ ]:
#도움: https://github.com/rickiepark/handson-gb/blob/main/Chapter04/Gradient_Boosting.ipynb

# 그래디언트 부스팅
- target을 예측하는 모델 외에, 잔차를 예측하는 모델을 더 만든다. 
- 초반의 잔차 예측 모델은 더 많이 들쭉

In [1]:
# pandas와 numpy 임포트하기
import pandas as pd
import numpy as np

# 경고 끄기
import warnings
warnings.filterwarnings('ignore')

In [3]:
df_bikes = pd.read_csv('../data/bike_rentals_cleaned.csv')
df_bikes.head()

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,1,1.0,0.0,1.0,0.0,6.0,0.0,2,0.344167,0.363625,0.805833,0.160446,985
1,2,1.0,0.0,1.0,0.0,0.0,0.0,2,0.363478,0.353739,0.696087,0.248539,801
2,3,1.0,0.0,1.0,0.0,1.0,1.0,1,0.196364,0.189405,0.437273,0.248309,1349
3,4,1.0,0.0,1.0,0.0,2.0,1.0,1,0.200000,0.212122,0.590435,0.160296,1562
4,5,1.0,0.0,1.0,0.0,3.0,1.0,1,0.226957,0.229270,0.436957,0.186900,1600


In [8]:
# 데이터를 X와 y로 나눕니다.
X_bikes = df_bikes.iloc[:,:-1]
y_bikes = df_bikes.iloc[:,-1]

# train_test_split를 임포트합니다.
from sklearn.model_selection import train_test_split

# 데이터를 훈련 세트와 테스트 세트로 나눕니다.
X_train, X_test, y_train, y_test = train_test_split(X_bikes, y_bikes, random_state=2)

X_train.head(), y_train.head()

(     instant  season   yr  mnth  holiday  weekday  workingday  weathersit  \
 203      204     3.0  0.0   7.0      0.0      6.0         0.0           1   
 634      635     4.0  1.0   9.0      0.0      3.0         1.0           1   
 544      545     3.0  1.0   6.0      0.0      4.0         1.0           1   
 12        13     1.0  0.0   1.0      0.0      4.0         1.0           1   
 171      172     3.0  0.0   6.0      0.0      2.0         1.0           2   
 
          temp     atemp       hum  windspeed  
 203  0.849167  0.804287  0.500000   0.131221  
 634  0.635000  0.596613  0.630833   0.244400  
 544  0.749167  0.675512  0.422500   0.171650  
 12   0.165000  0.150883  0.470417   0.301000  
 171  0.680833  0.637646  0.770417   0.171025  ,
 203    3285
 634    7733
 544    6879
 12     1406
 171    4835
 Name: cnt, dtype: int64)

In [9]:
# DecisionTreeRegressor를 임포트합니다.
from sklearn.tree import DecisionTreeRegressor

# mean_squared_error를 임포트합니다.
from sklearn.metrics import mean_squared_error as MSE

# step1: Decision Tree 회귀 모델을 만들어서, target을 예측하게 훈련시키기
tree_1 = DecisionTreeRegressor(max_depth=2, random_state=2)
tree_1.fit(X_train, y_train)
y_train_pred = tree_1.predict(X_train) # 훈련셋에 대해 그대로 예측한 결과
y2_train = y_train - y_train_pred # 훈련셋에 대한 잔차 (예측값 - 실제값)를 계산한 것

# step2: Decision Tree (2번쨰) 모델을 만들어서, 이전 모델에서의 잔차를 예측하게 훈련시키기
tree_2 = DecisionTreeRegressor(max_depth=2, random_state=2)
tree_2.fit(X_train, y2_train)
y2_train_pred = tree_2.predict(X_train)
y3_train = y2_train - y2_train_pred

# step3: Decision Tree (3번쨰) 모델을 만들어서, 이전 모델에서의 잔차를 예측하게 훈련시키기
tree_3 = DecisionTreeRegressor(max_depth=2, random_state=2)
tree_3.fit(X_train, y3_train)

# step4: 실제 예측 시에는 첫모델(target 예측) + 2모델(잔차 예측) + 3모델(잔차 예측)한 것을 더한다.
y_pred = tree_1.predict(X_test) + tree_2.predict(X_test) + tree_3.predict(X_test)


# 평균 제곱근 오차를 계산합니다.
MSE(y_test, y_pred)**0.5

911.0479538776444

In [12]:
from sklearn.metrics import mean_squared_error

# 첫번째 모델에서 타겟 대신 타겟과 타겟 평균 사이의 차이를 예측
res_1 = y_train - np.mean(y_train)
tree_1.fit(X_train, res_1)
pred_1 = tree_1.predict(X_train)

# 두번째 모델부터는 마찬가지로 잔차를 예측
res_2 = y_train - pred_1
tree_2.fit(X_train, res_2)
pred_2 = tree_2.predict(X_train)

# 세번째 모델부터는 마찬가지로 잔차를 예측
res_3 = y_train - (pred_1 + pred_2)
tree_3.fit(X_train, res_3)

# 실제 예측 결과는 첫모델(target 예측) + 2모델(잔차 예측) + 3모델(잔차 예측)한 것을 더한다.
pred_all = tree_1.predict(X_test) + tree_2.predict(X_test) + tree_3.predict(X_test)
MSE(y_test, pred_all)**0.5 #mean_squared_error(y_test, pred_all, squared=False)

911.0479538776432

In [7]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(max_depth=2, n_estimators=3, 
                                random_state=2, learning_rate=1.0)

gbr.fit(X_train, y_train)

# 테스트 데이터에 대한 예측을 만듭니다.
y_pred = gbr.predict(X_test)

# 평균 제곱근 오차를 계산합니다.
MSE(y_test, y_pred)**0.5

911.0479538776439

In [13]:
import matplotlib.pyplot as plt
import seaborn as sns

params = {'subsample':[0.65, 0.7, 0.75],
          'n_estimators':[300, 500, 1000],
          'learning_rate':[0.05, 0.075, 0.1]}

# RandomizedSearchCV를 임포트합니다.
from sklearn.model_selection import RandomizedSearchCV

gbr = GradientBoostingRegressor(max_depth=3, random_state=2)


# RandomizedSearchCV를 초기화합니다.
rand_reg = RandomizedSearchCV(gbr, params, n_iter=10, 
                              scoring='neg_mean_squared_error', 
                              cv=5, n_jobs=-1, random_state=2)

# X_train와 y_train로 rand_reg를 훈련합니다.
rand_reg.fit(X_train, y_train)

# 최상의 모델을 추출합니다.
best_model = rand_reg.best_estimator_

# 최상의 매개변수를 추출합니다.
best_params = rand_reg.best_params_

# 최상의 매개변수를 출력합니다.
print("최상의 매개변수:", best_params)

# 최상의 점수를 계산합니다.
best_score = np.sqrt(-rand_reg.best_score_)

# 최상의 점수를 출력합니다.
print("훈련 점수: {:.3f}".format(best_score))

# 테스트 세트에 대한 예측을 만듭니다.
y_pred = best_model.predict(X_test)

# 평균 제곱근 오차를 계산합니다.
rmse_test = MSE(y_test, y_pred)**0.5

# 평균 제곱근 오차를 출력합니다.
print('테스트 세트 점수: {:.3f}'.format(rmse_test))

최상의 매개변수: {'subsample': 0.65, 'n_estimators': 300, 'learning_rate': 0.05}
훈련 점수: 636.200
테스트 세트 점수: 625.985


In [14]:
gbr = GradientBoostingRegressor(max_depth=3, n_estimators=1600, 
                                subsample=0.75, learning_rate=0.02, 
                                random_state=2)
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)
MSE(y_test, y_pred)**0.5

596.9544588974487